In [18]:
%load_ext autoreload
%autoreload 2

In [11]:
def decorator(func):
    print("Inside decorator")
     
    def wrapper(*args, **kwargs):
         
        # code functionality here
        print("Inside inner function")
        print("I like")
        # print(kwargs['message'])
        func(*args, **kwargs)
         
    # returning inner function    
    return wrapper

tmp = "hello"
@decorator
def my_func():
    print("Inside actual function")

x

Inside decorator


1

In [76]:
from final_project.Trading_Project.utils.errors_handler import error_tracking_decorator, get_root_cause_error

def plus(a, b):
    return a + b


@error_tracking_decorator
def my_func():
    result = plus(1,2)

s = my_func()
s

'Error in function my_func \n===================================================\nFilename: /tmp/ipykernel_130327/762026832.py,\nFunction name: my_func, params: {}\n\t-----\n\tLine: 9,     result = plus(\'1\',2)\n \t-----\n\tNote: This error is from your code\n===================================================\nFilename: /tmp/ipykernel_130327/762026832.py,\nFunction name: plus, params: {\'a\': \'1\', \'b\': 2}\n\t-----\n\tLine: 4,     return a + b\n \t-->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str \n\t-----\n\tNote: This error is from your code\n'

In [ ]:
ss = [('\n'.join([ssi for ssi in s.split('\n') if '-----' not in ssi and 'Note' not in ssi and 'Filename' not in ssi])).strip().replace('\t', '') for s in s.split("===================================================")]
ss


In [54]:

processed_strings = [process_string(clean_string(string)) for string in s.split("===================================================")]
processed_strings


['Error in function my_func',
 "Function name: my_func, params: {}\nLine: 9,     result = plus('1',2)",
 'Function name: plus, params: {\'a\': \'1\', \'b\': 2}\nLine: 4,     return a + b\n -->ROOT CAUSE: TypeError: can only concatenate str (not "int") to str ']

In [69]:

print("import backtrader as bt\n\nclass CustomIchimokuIndicator(bt.Indicator):\n    \"\"\"\n    Custom Ichimoku Cloud indicator.\n    \"\"\"\n    lines = (\"tenkan\", \"kijun\", \"senkou_a\", \"senkou_b\")\n    params = ((\"tenkan_period\", 9), (\"kijun_period\", 26), (\"senkou_period\", 52))\n\n    def __init__(self):\n        self.addminperiod(self.params.tenkan_period)\n        self.addminperiod(self.params.kijun_period)\n        self.addminperiod(self.params.senkou_period)\n\n    def next(self):\n        high = self.data.high.get(size=self.params.tenkan_period)\n        low = self.data.low.get(size=self.params.tenkan_period)\n        close = self.data.close.get(size=self.params.tenkan_period)\n        tenkan = (max(high) + min(low)) / 2\n        kijun = (max(high.get(size=self.params.kijun_period)) + min(low.get(size=self.params.kijun_period))) / 2\n        senkou_a = (tenkan + kijun) / 2\n        senkou_b = (max(high.get(size=self.params.senkou_period)) + min(low.get(size=self.params.senkou_period))) / 2\n        self.lines.tenkan[0] = tenkan\n        self.lines.kijun[0] = kijun\n        self.lines.senkou_a[0] = senkou_a\n        self.lines.senkou_b[0] = senkou_b\n\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.ichimoku = CustomIchimokuIndicator()\n\n    def execute(self):\n        \"\"\"\n        Define the trading logic based on the Ichimoku Cloud indicator.\n\n        Returns:\n        int: Trading signal: 1 (long), -1 (sell), or None if no signal.\n        \"\"\"\n        if self.ichimoku.tenkan[0] > self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] > self.ichimoku.senkou_b[0]:\n            return 1  # Long signal\n        elif self.ichimoku.tenkan[0] < self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] < self.ichimoku.senkou_b[0]:\n            return -1  # Short signal\n        return None  # No signal\n")

import backtrader as bt

class CustomIchimokuIndicator(bt.Indicator):
    """
    Custom Ichimoku Cloud indicator.
    """
    lines = ("tenkan", "kijun", "senkou_a", "senkou_b")
    params = (("tenkan_period", 9), ("kijun_period", 26), ("senkou_period", 52))

    def __init__(self):
        self.addminperiod(self.params.tenkan_period)
        self.addminperiod(self.params.kijun_period)
        self.addminperiod(self.params.senkou_period)

    def next(self):
        high = self.data.high.get(size=self.params.tenkan_period)
        low = self.data.low.get(size=self.params.tenkan_period)
        close = self.data.close.get(size=self.params.tenkan_period)
        tenkan = (max(high) + min(low)) / 2
        kijun = (max(high.get(size=self.params.kijun_period)) + min(low.get(size=self.params.kijun_period))) / 2
        senkou_a = (tenkan + kijun) / 2
        senkou_b = (max(high.get(size=self.params.senkou_period)) + min(low.get(size=self.params.senkou_period))) / 2
        self.lines.tenk

In [70]:
import backtrader as bt

class CustomIchimokuIndicator(bt.Indicator):
    """
    Custom Ichimoku Cloud indicator.
    """
    lines = ("tenkan", "kijun", "senkou_a", "senkou_b")
    params = (("tenkan_period", 9), ("kijun_period", 26), ("senkou_period", 52))

    def __init__(self):
        self.addminperiod(self.params.tenkan_period)
        self.addminperiod(self.params.kijun_period)
        self.addminperiod(self.params.senkou_period)

    def next(self):
        high = self.data.high.get(size=self.params.tenkan_period)
        low = self.data.low.get(size=self.params.tenkan_period)
        close = self.data.close.get(size=self.params.tenkan_period)
        tenkan = (max(high) + min(low)) / 2
        kijun = (max(high.get(size=self.params.kijun_period)) + min(low.get(size=self.params.kijun_period))) / 2
        senkou_a = (tenkan + kijun) / 2
        senkou_b = (max(high.get(size=self.params.senkou_period)) + min(low.get(size=self.params.senkou_period))) / 2
        self.lines.tenkan[0] = tenkan
        self.lines.kijun[0] = kijun
        self.lines.senkou_a[0] = senkou_a
        self.lines.senkou_b[0] = senkou_b


class BackTestStrategy(BaseStrategy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ichimoku = CustomIchimokuIndicator()

    def execute(self):
        """
        Define the trading logic based on the Ichimoku Cloud indicator.

        Returns:
        int: Trading signal: 1 (long), -1 (sell), or None if no signal.
        """
        if self.ichimoku.tenkan[0] > self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] > self.ichimoku.senkou_b[0]:
            return 1  # Long signal
        elif self.ichimoku.tenkan[0] < self.ichimoku.kijun[0] and self.ichimoku.senkou_a[0] < self.ichimoku.senkou_b[0]:
            return -1  # Short signal
        return None  # No signal

In [60]:
from final_project.Trading_Project.data_loader import load_stock_data

data = [bt.feeds.PandasData(
                dataname=load_stock_data(ticker='AAPL', period="1y"), datetime="Date", 
                timeframe=bt.TimeFrame.Minutes)]



[*********************100%%**********************]  1 of 1 completed


In [77]:
class MyBuySell(bt.observers.BuySell):
    plotlines = dict(
        buy=dict(marker='$\u21E7$', markersize=12.0),
        sell=dict(marker='$\u21E9$', markersize=12.0)
    )

class CelebroCreator:
    def __init__(self, strategy, list_of_data, stake=100, cash=20000):
        self.cerebro = bt.Cerebro(cheat_on_open=True)

        self.cerebro.addstrategy(strategy)
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
        self.cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
        self.cerebro.addobserver(bt.observers.Value)

        for data in list_of_data:
            self.cerebro.adddata(data)
        self.cerebro.broker.set_cash(cash)
        bt.observers.BuySell = MyBuySell
        self.strats = self.cerebro.run()


    def show(self):
        print("Final Portfolio Value: %.2f" % self.cerebro.broker.getvalue())
        print("Total point return: ", (self.cerebro.broker.getvalue() - self.cerebro.broker.startingcash))

        try:
            sharpe_ratio = self.strats[0].analyzers.SharpeRatio.get_analysis()['sharperatio']
        except:
            print('No Buy/Sell Signal! No Sharpe Ratio!')
        # Plot the results
        figs = self.cerebro.plot(
            iplot=False, 
            # style="pincandle", 
            # width=60 * 10, height=40 * 10,
            figsize=(100, 80),
            # sharpe_ratio=sharpe_ratio
        )

def check_valid_code(strategy, list_data):
    
    @error_tracking_decorator
    def extract_error_message(strategy, list_data):
        CelebroCreator(strategy,list_data)
        
    message = extract_error_message(BackTestStrategy, data)
    root_error = get_root_cause_error(message) if message else ""
    return root_error

check_valid_code(BackTestStrategy, data)

["AttributeError: 'array.array' object has no attribute 'get'",
 'Function name: next, params: {\'high\': "array(\'d\', [178.4199981689453, 1...", \'low\': "array(\'d\', [174.8000030517578, 1...", \'close\': "array(\'d\', [177.14999389648438, ...", \'tenkan\': 172.0449981689453}',
 'Line: 20,         kijun = (max(high.get(size=self.params.kijun_period)) + min(low.get(size=self.params.kijun_period))) / 2']